In [1]:
import cv2
import numpy as np
import math
import os

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets

#  <span style ="text-decoration: underline">Sign Language Interpreter</span>

### <span style ="font-family:Times New Roman"><span style ="color:blue">**The sign language interpreter is an application which functions as a tool that can break down the communication barrier between the hard-of-hearing and the general public.** </span></span>

![SIGN LANGUAGE](images/signlangbg.jpg)

In [3]:
image = cv2.imread("images/test/hello/1.jpg")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
histogramtest = cv2.calcHist([gray_image], [0],  None, [256], [0, 256])

In [11]:
image = cv2.imread('images/test/no/153.jpg') 
gray_image1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
histogramno = cv2.calcHist([gray_image1], [0],  None, [256], [0, 256])

In [5]:
image = cv2.imread('images/test/hello/101.jpg') 
gray_image2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
histogramhello = cv2.calcHist([gray_image2], [0],  
                          None, [256], [0, 256]) 

In [6]:
image = cv2.imread('images/test/yes/100.jpg') 
gray_image2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
histogramyes = cv2.calcHist([gray_image2], [0],  
                          None, [256], [0, 256]) 

In [7]:
image = cv2.imread('images/test/I/249.jpg') 
gray_image2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
histogrami = cv2.calcHist([gray_image2], [0],  
                          None, [256], [0, 256]) 

In [8]:
image = cv2.imread('temp/tempwhite.png') 
gray_image2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
histogramwhite = cv2.calcHist([gray_image2], [0],  
                          None, [256], [0, 256]) 

In [9]:
def calculate(histogramweb):
    c1, c2, c3, c4, c5 = 0,0,0,0,0
    
    i = 0
    while i<len(histogramweb) and i<len(histogramno): 
        c1+=(histogramweb[i]-histogramno[i])**2
        i+= 1
    c1 = c1**(1 / 2) 
    
    j = 0
    while j<len(histogramweb) and j<len(histogramhello): 
        c2+=(histogramweb[j]-histogramhello[j])**2
        j+= 1
    c2 = c2**(1 / 2)
    
    z = 0
    while z<len(histogramweb) and z<len(histogramyes): 
        c3+=(histogramweb[z]-histogramyes[z])**2
        z+= 1
    c3 = c3**(1 / 2)   
    
    w = 0
    while w<len(histogramweb) and w<len(histogrami): 
        c4+=(histogramweb[w]-histogrami[w])**2
        w+= 1
    c4 = c4**(1 / 2) 
    
    a = 0
    while a<len(histogramweb) and a<len(histogramwhite): 
        c5+=(histogramweb[a]-histogramwhite[a])**2
        a+= 1
    c5 = c5**(1 / 2) 
    
    clist = [c1,c2,c3,c4,c5]
    minimum = min(clist)
    if c1 == minimum:
        string = "sign is no"
    elif c2 == minimum:
        string = "sign is hello"
    elif c3 == minimum:
        string = "sign is yes"
    elif c4 == minimum:
        string = "sign is i"
    else:
        string = "Unrecognized Pattern"
    return string

In [12]:
def getwebcam(arg):
    cap = cv2.VideoCapture(0)
    while True:

        try:
            ret, frame = cap.read()
            frame=cv2.flip(frame,1)
            kernel = np.ones((3,3),np.uint8)
            roi=frame[100:300, 100:300]
            cv2.rectangle(frame,(100,100),(300,300),(0,255,0),0)
            hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            lower_skin = np.array([0,20,70], dtype=np.uint8)
            upper_skin = np.array([20,255,255], dtype=np.uint8)
            mask = cv2.inRange(hsv, lower_skin, upper_skin)
            mask = cv2.dilate(mask,kernel,iterations = 4)

            #image is blurred using GBlur
            #mask = cv2.GaussianBlur(mask,(5,5),100)
            ret,thresh = cv2.threshold(mask,0,255,cv2.THRESH_BINARY)
            contours,hierarchy= cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            cnt = max(contours, key = lambda x: cv2.contourArea(x))
            epsilon = 0.0005*cv2.arcLength(cnt,True)
            approx= cv2.approxPolyDP(cnt,epsilon,True)
            hull = cv2.convexHull(cnt)


            hull = cv2.convexHull(approx, returnPoints=False)
            defects = cv2.convexityDefects(approx, hull)
            count_defects = 0

            for i in range(defects.shape[0]):
                s,e,f,d = defects[i,0]

                path = 'temp'
                cv2.imwrite(os.path.join(path , 'temp.png'), thresh)
                image = cv2.imread("temp/temp.png")
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
                histogramweb = cv2.calcHist([gray_image], [0],  None, [256], [0, 256])
                x = calculate(histogramweb)
                #print(x)
                cv2.putText(frame, x, (0,50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)

            cv2.imshow('mask',mask)
            cv2.imshow('frame',frame)
            cv2.imshow("gray", gray_image)
            cv2.imshow('threshold',thresh)
        except:
            pass


        if cv2.waitKey(1) == ord('q'):
            break

    cv2.destroyAllWindows()
    cap.release()

button_download = widgets.Button(description = 'CLICK ME !!', button_style='primary')  
button_download.on_click(getwebcam)
display(button_download)

Button(button_style='primary', description='CLICK ME !!', style=ButtonStyle())